In [1]:
import sys
from os.path import isfile
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from scipy import stats
import random
import logging
from add_spikes import add_spikes
from shift_spectrum import shift_spectrum
from get_redshift import get_redshift
import os

In [2]:
# -- Logging info
logging.basicConfig(filename='modifications.txt', level=logging.INFO,
                    format='%(asctime)s - %(message)s', filemode='w')

In [3]:
def modify(spectrum, sn_name, delay_phase):
    # --Error handling
    if not isinstance(spectrum, np.ndarray):
        raise TypeError(f"Error: Expected 'np.ndarray', got {type(spectrum).__name__}")
    if not isinstance(sn_name, str):
        raise TypeError(f"Error: Expected 'str', got {type(sn_name).__name__}")
    if not isinstance(delay_phase, float):
        raise TypeError(f"Error: Expected 'float', got {type(delay_phase).__name__}")

    # --Copy spectrum and log basic information
    modified_spectrum = spectrum.copy()
    logging.info(f'Object: {sn_name}, phase delay: {delay_phase}')

    # --Prepare modification details
    modification_details = {
        'Object': sn_name,
        'Phase Delay': delay_phase,
        'Modification Type': '',
        'Redshift Applied': False,
        'Spikes Added': False
    }

    # --Apply modifications
    modifications = random.randint(0, 3)
    if modifications == 0:
        logging.info('---Modifications: None')
        modification_details['Modification Type'] = 'None'
    else:
        try:
            redshift = get_redshift(sn_name)  # Intentar obtener el redshift
            modification_details['Redshift Applied'] = True
        except Exception as e:
            logging.warning(f"Warning: No se encontró redshift para {sn_name}. Detalle: {e}")
            redshift = 0.001  # Valor de redshift alternativo si no se encuentra el original
            logging.info(f'Usando redshift alternativo: {redshift}')

        if modifications == 1:
            logging.info('---Modifications: Shift spectra on x axis only')
            modified_spectrum = shift_spectrum(spectrum, redshift)
            modification_details['Modification Type'] = 'Shift Spectrum'
        elif modifications == 2:
            logging.info('---Modifications: Add spikes only')
            modified_spectrum = add_spikes(spectrum, redshift)
            modification_details['Modification Type'] = 'Add Spikes'
            modification_details['Spikes Added'] = True
        elif modifications == 3:
            logging.info('---Modifications: Shift spectra on x axis AND add spikes')
            modified_spectrum = add_spikes(shift_spectrum(spectrum, redshift), redshift)
            modification_details['Modification Type'] = 'Shift and Add Spikes'
            modification_details['Spikes Added'] = True

    # Convert details dictionary to a DataFrame
    modification_table = pd.DataFrame([modification_details])

    return modified_spectrum, modification_table

In [4]:
np.random.seed(3312)
url = "https://github.com/juanserrano90/codelatam/raw/main/Data/data/sn_data.parquet"
df_raw = pd.read_parquet(url)
wavelength = np.array([float(c) for c in df_raw.columns[5:]])
column_names = df_raw.columns.values.tolist()
column_names = np.array(column_names[5:]).astype(float)


In [6]:
data_aumentation_plan = pd.DataFrame(columns=['Type', 'N° of Copys'])

data_aumentation_plan['Type'] = ['Ia-91T', 'Ia-91bg', 'Ia-pec']
data_aumentation_plan['N° of Copys'] = [172, 306 , 500]

In [7]:
df_raw['SN Subtype'].value_counts()

Ia-norm     2387
Ia-91T       398
IIb          328
Ic-broad     279
Ib-norm      270
Ia-91bg      264
Ic-norm      263
II-pec       241
IIP          176
Ia-pec       141
Iax           68
IIn           64
Ic-pec        33
Ibn           31
Ia-csm        30
Ib-pec        15
IIL           13
Name: SN Subtype, dtype: int64

In [8]:
#np.random.seed(1)

modified = []
modification_tables = []


for i in data_aumentation_plan['Type']:
    print(f"Procesando tipo: {i}")
    
    # Selecciona el subtipo a aumentar
    df_loc = df_raw[df_raw['SN Subtype'] == i]
    
    # Selecciona el número de copias necesarias
    n = data_aumentation_plan[data_aumentation_plan['Type'] == i]['N° of Copys'].values[0]

    # Bucle para asegurar el número de copias deseado
    while len(modification_tables) < n:
        # Selecciona una muestra
        df_loc_sampled = df_loc.sample(n=1, replace=True)
        
        # Aplica las modificaciones:
        for index, row in df_loc_sampled.iterrows():
            sn_name = index
            try:
                # Obtiene el valor de 'Spectral Phase'
                delay_phases = row['Spectral Phase']
                
                # Intenta obtener la muestra correspondiente de df_raw
                sample = df_raw.loc[(df_raw.index == sn_name)].iloc[:, 5:].values[0]

                # Modifica el espectro y guarda la tabla de modificaciones
                mod_spectrum, mod_table = modify(sample, sn_name, delay_phases)
                modified.append(mod_spectrum)
                modification_tables.append(mod_table)

            except IndexError as e:
                print(f"Error: No se encontró un valor para la supernova {sn_name}. Detalle: {e}")
            except KeyError as e:
                print(f"Error: Columna no encontrada en la fila de {sn_name}. Detalle: {e}")
            except Exception as e:
                print(f"Error inesperado para {sn_name}. Detalle: {e}")



    

Procesando tipo: Ia-91T
Error inesperado para sn2001eh. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1999dq. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1999dq. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2001eh. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1999aa. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1998es. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1998es. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1998es. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1999aa. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Erro

Error inesperado para sn1991bg. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2005bl. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2005ke. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn1991bg. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Procesando tipo: Ia-pec
Error inesperado para sn2000cx. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2000cx. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2000cx. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2000cx. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Error inesperado para sn2000cx. Detalle: operands could not be broadcast together with shapes (1024,) (2,) 
Erro